In [1]:
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"]=(12, 12)
%matplotlib inline
import numpy as np
import random
import numpy as np
from scipy.spatial.distance import euclidean
from collections import defaultdict
from itertools import combinations


In [ ]:
df=pd.read_csv(read the input file here)

In [ ]:
check for null and missing values in the dataset. Devise a plan re how to fill those values

df.head()

In [ ]:
Get the ID column separately and drop it in the dataset

salesresp=df["ID"]

In [ ]:
df.drop('ID', axis=1, inplace=True)

In [ ]:
X will be a numpy array. Remember that X here has been already normalized in order to get rid of the curse of 
dimensionality

X=df.values

In [ ]:
#used sklearn kmeans ELBOW method to find the optimal value for k
from sklearn.cluster import kmeans
wcss=[]

#we, most often, assume the max number of clsuter would be 10
#can judge the number of clusters by averaging

for i in range(1, 11):
    kmeans=KMeans(n_clusters=i, init='k-means++', random_state=0)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)#inertia is the formula used to segregate the data points into clusters
    
    

In [ ]:
#visualizing the ELBOW method to get the optimal value of k
plt.plot(range(1,11),wcss)
plt.title('The Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('Percent of variance explained_wcss')
plt.show()

Observation: sample
the second elbow falls at 3 so will go with k=3 clusters


In [ ]:
Coding kmeans from scratch:
    
def kmeans(X, k, max_iter=1000):
	centers=[tuple(c) for c in random.sample(X.tolist(), k)]#it randomly picks k elements from the input array X
	#X.tolist()-this is because X is a numpy array and so have to convert it into a list of values


	for i in range(max_iter):
		clusters=defaultdict(list)
		clusters_repid=defaultdict(list)
		repid=[]
		for value in X:
			ind=np.where(X==value)
			distance=[euclidean(value, x0) for x0 in centers]#find the euclidean distance between each value in X and each point in centers
			result=centers[np.argmin(distance)]#np.argmin returns the index of the minimum distance. find the value of the index in centers and append it to clusters cause that is the centroid
			clusters[result].append(value)
			clusters_repid[result].extend(list(ind))#list(ind) is because ind is also a numpy array returned by
			#statement np.where
			
			# get the index of value with re to the dataframe and use the index finally to pull the SalesRep_ID value
			# from the dataframe

		new_centers=[]
		for c, p in clusters.items():
			new_centers.append(tuple(np.mean(p, axis=0)))

		if set(new_centers)==set(centers):#if the earlier version of center the same as the newly created centers then break the loop
			break

		centers=new_centers#if not break then assign the newly created centers to the earlier version, i.e., centers and continue with the next iteration

	return clusters, clusters_repid


In [ ]:
clusters, clusters_repid=kmeans(X, k=3, max_iter=5)#X here is a numpy array

In [ ]:
Can use this to figure out the optimal value for k. This is just a part of a whole coding...this formula is what we use
to calculate at which value of k the error is the least

def sum_squared_error(clusters):
	sum_error=0
	for cen, point in clusters.items():
		for i in point:
			sum_error+=euclidean(i, cen)**2
	return sum_error

In [ ]:
def turn_clusters_labels(clusters, clusters_repid, salesrep):
	l=0
	
	new_X=[]
	labels=[]
	for_repid=[]

	for center, point in clusters.items():
		for pt in point:
			new_X.append(pt)
	
			labels.append([l])#not sure whether [l] is necessary here

		l+=1

	for c, p in clusters_repid.items():
		for pts in p:
			for_repid.append(salesrep.loc[pts, 'SALESREP_ID'])

	return labels, new_X, for_repid#ideally it would be better to return lists so it is easier to use later in the
	#process especially to loop through and get salesrepid from a pandas dataset. it's not an easy task to do all this
	#using numpy arrays 

In [ ]:
labels, new_X, for_repid=turn_clusters_labels(clusters, clusters_repid, salesrep)

In [ ]:
new_X#is a numpy arraty

X_new_fair=np.array(new_X)

In [ ]:
To plot list of values with varying lengths, convert the list of values into a dataframe columns

df_visuals=pd.DataFrame.from.dict({"labels":labels, "X_values":X_new_fair[:, 0].tolist(), "SalesRepID":for_repid[0].tolist()}, orient='index').T

important things to note:
X_new_fair has two columns that can be accessed as X_new_fair[:, 0] and X_new_fair[:, 1]
Likewise for_repid has two columns and that can be accessed as for_repid[0] and for_repid[1]
This is primarily because of how the random samples have been formed




In [ ]:
if u want to delete the square brackets of lists from the values in a column

df_visuals["some column"]=df_visuals["some column"].str[0]

In [ ]:
Sample:
    
import seaborn as sns; sns.set()
cmap=sns.cubehelix_palette(dark=0.8, light=4.8, as_cmap=True)
ax=sns.scatterplot(x='X_values', y='SalesRepID', hue='labels', palette="inferno",data=df_visuals, sizes=(1300, 2500))
ax.set_title('Clusters of similar SalesRepID ...')


In [ ]:
In case if you have to plot all 2D plots with re to every feature in the input dataset - check the 2D plot from 
galvanize repo




In [ ]:
Another way to plot

ax=df_visuals.plot.scatter(x='X_values', y='SalesRepID', s=330, c='labels', colormap='viridis')
plt.title('')
plt.xlabel('X_values')
plt.ylabel('SalesRepID')
plt.show()